# Quelques programmes utilisés lors du Hackathon

Voici une partie des programmes utilisés lors du Hackathon. Le groupe disposait d'un programme de matching plus élaboré. Ce programme était capable d'identifier certaines structures des chaines de caractères à partir de chaines de markov cachées. Le travail présenté était destiné à enrichir cette approche. Il n'est pas destiné à fonctionner de façon autonome. Ceci explique ses piètres performances. 

Trois directions ont été explorés :
1. Le webscraping des pages jaunes afin de trouver certaines adresses
2. Le codage de l'activité (code NAF sur 2 positions) à partir de l'activité déclarée au recensement.
3. L'identification des groupes de raisons sociales les plus problématiques (éducation, défence etc.)

Les points 2 et 3 sont sans doute les plus intéressants. 

## Prérequis

### Importation des librairies requises

In [ ]:
#Importation des librairies requises

import re
import string as txt
import nltk
from nltk.stem.snowball import FrenchStemmer #import the French stemming library
from nltk.corpus import stopwords #import stopwords from nltk corpus
import operator
import unidecode

### Quelques fonctions de manipulation de chaine de caractères

Une première fonction de nettoyage de données textuelles :

In [ ]:
def clean_text(string):
    string = str(string)
    # suppression des accents
    string = unidecode.unidecode(string)
    # Remplace les signes de ponctuations
    for c in txt.punctuation :
        string = re.sub(re.escape(c),' ',string)
    string = string.strip()
    # remplacement des sigles
    s = re.findall(r'\s(\w\s\w)\s',string)
    if len(s)>0 : string = re.sub(' '+s[0]+' ',re.sub(' ','',s[0]),string)
    # Mise en majuscules
    string = string.upper()
    # Suppression des espaces inutiles
    string = string.strip()  
    # Remplace les tabluations, sauts de lignes, etc. par des espaces.
    string = re.sub('\n|\r|\t|\xa0', ' ', string)
    # Retire les ' .' (séparateurs utilisés dans `soup.get_text`) inappropriés.
    string = re.sub('^\.', '', string.replace(' .', ''))
    # Retire les espaces inappropriés.
    return string

Une fonction de comptage du nombre de mots dans une colonne (vecteur ?) au format texte :

In [ ]:
def comptage_mots(colonne):
    
    # concatenation de la colonne 
    words = " ".join(colonne)
    # Nettoyage sommaire des donnees textuelles
    words = re.sub('\n','',words)
    words = re.sub('[^A-Z\s]|','',words)
    words = words.strip()
    tokens = nltk.word_tokenize(words,language='french')

    # Nettoyage de la chaine de caracteres des stopwords
    clean_tokens = tokens[:]
    for token in tokens:
        if token.lower() in stopwords.words('french'):
            clean_tokens.remove(token)
    # Affichage pour verification
    print(clean_tokens[:100])    

    # Comptage du nombre de mots
    fidist2 = nltk.FreqDist(clean_tokens)

    # tri des mots les plus frequents
    fidist2 = sorted(fidist2.items(), reverse=True, key=operator.itemgetter(1))
    
    return fidist2

### Importation des données

In [1]:
# gestion des tables de donnees
import pandas as pd
import numpy as numpy


# Importation des donnees du RP
rp = pd.read_csv("/Users/stephaniehimpens/Documents/Hackathon/donnees/Donnees/rp_final_2014.csv",sep=";",dtype=str)
print("Jeux de données chargé ; " + str(rp.shape[0]) + " lignes, " + str(rp.shape[1]) + " colonnes.")

# Recodage des donnees manquantes
rp.loc[rp.NUMVOI_X.isnull(),"NUMVOI_X"]=""
rp.loc[rp.BISTER_X.isnull(),"BISTER_X"]=""
rp.loc[rp.TYPEVOI_X.isnull(),"TYPEVOI_X"]=""
rp.loc[rp.NOMVOI_X.isnull(),"NOMVOI_X"]=""
rp.loc[rp.CPLADR_X.isnull(),"CPLADR_X"]=""

rp.NUMVOI_X = [str(int("0" + word)) for word in rp.NUMVOI_X]
rp.loc[rp.NUMVOI_X=='0',"NUMVOI_X"]=''


# Concatenation des champs de l adresse
rp.ADR_FULL = rp.NUMVOI_X + " " + rp.BISTER_X + " " + rp.TYPEVOI_X + " "+ rp.NOMVOI_X + " " + rp.CPLADR_X
sup_blanc = lambda x:x.strip()
rp.ADR_FULL = rp.ADR_FULL.map(sup_blanc)

Jeux de données chargé ; 30876 lignes, 52 colonnes.


Afin de minimiser le temps de traitement, l'appariement a été réalisé sur un échantillon de 500 observations du RP.

In [2]:
# tirage aleatoire
import random as rd

#tirage aleatoire des donnees du rp
id_ech = numpy.random.choice(range(rp.shape[0]),size=500,replace=False)
rp = rp.ix[id_ech,]

/Users/stephaniehimpens/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [ ]:
# Nettoyage des RS du recensement  
# remplacement des codes communes
for index,row in rp.iterrows():
    depcom = str(row.DEPCOM_CODE)
    depcom = str(row.CLT_X) + ' ' + depcom[0:2]
    row.RS_X = re.sub(depcom,'',str(row.RS_X))   
    row.RS_X = re.sub(str(row.CLT_X),'',str(row.RS_X))
    tags = tagger.tag_text(row.RS_X)
    c=''
    for t in range(len(tags)) :
       if t=='' : c = c + " " + str(tags[t]).split('\t')[2]
       c = c.strip()
    if len(c)>0 : row.RS_X = c
    rp.ix[index,"RS_X"] = row.RS_X
rp.RS_X = [clean_text(word) for word in rp.RS_X]
rp.CLT_X = [clean_text(word) for word in rp.CLT_X]
rp.CLT_X = [re.sub('ST ','SAINT ',word) for word in rp.CLT_X]
rp.CLT_X = [re.sub(r'[0-9]','',word).strip() for word in rp.CLT_X]
rp.CLT_X = [re.sub(r'[0-9]','',word).strip() for word in rp.CLT_X]
rp.CLT_X = [re.sub('NAN','',word).strip() for word in rp.CLT_X]

# Remplacements specifiques
rp.RS_X = [re.sub('VILLE','COMMUNE'+str(rp.CLT_X),word) for word in rp.RS_X]
rp.RS_X = [re.sub('MAIRIE','COMMUNE'+str(rp.CLT_X),word) for word in rp.RS_X]
rp.RS_X = [re.sub('LBV YVES ROCHER','YVES ROCHER',word) for word in rp.RS_X]
rp.RS_X = [re.sub('CHU ','HOPITAL',word) for word in rp.RS_X]

## Le webscraping des pages jaunes à partir de la raison sociale et de la commune

Une fonction de webscraping a été écrite mais elle n'a finalement pas été utilisée. Les pages jaunes ont été bloqués pendant une bonne partie du Hackathon. 

In [ ]:
# Installation des librairies tierces utilisées.
import pip
for pkg in ['requests', 'bs4']:
    pip.main(['install', pkg])
from bs4 import BeautifulSoup
import requests
import re

Sélection des observations du recensement sans adresse. On n'en retient que 6. Il s'agit de tester le programme.

In [ ]:
testsorted = []
selection = rp.loc[rp.ADR_FULL=='']
selection = selection[:6]

Le programme boucle sur la table (récupération du nom de la commune (ou du numéro de département si non disponible) et de la raison sociale) et construit les requêtes pour le site des pages jaunes.
Les résultats sont récupérés dans une table panda (rp "enrichi").

In [ ]:
for index,row in selection.iterrows():
    # Mise en forme de la requete
    if row.CLT_X=='':
        if str(row.DLT_X)=='nan':
            test = 'https://www.pagesjaunes.fr/recherche/'+str(row.DEPCOM_CODE[:2])+'/'+row.RS_X
        else :
            test = 'https://www.pagesjaunes.fr/recherche/'+str(row.DLT_X)+'/'+row.RS_X
    else :
        test = 'https://www.pagesjaunes.fr/recherche/'+str(row.CLT_X)+'/'+str(row.RS_X)
    test = str(test)
    test = re.sub("\[","",test)
    test = re.sub("\]","",test)
    test = re.sub("'","",test)
    test = re.sub('"','',test)
    
    # Interrogation des pages jaunes
    response = requests.get(test)
    text = re.sub('denomination-links pj-link','denomination-links pj-lb pj-link',response.text)
    print(response.status_code)
    if response.status_code != 200:
        print('Échec de la requête: statut %s.' % response.status_code)

    # Mise en forme des donnees textuelles
    soup = BeautifulSoup(text, 'html.parser')
    rsp=[]
    adrp=[]
    actp=[]
    supp=[]
    for p in soup.find_all('article'):
        a = p.find_all('a',{'class','denomination-links pj-lb pj-link'})
        if len(a)==0:
            a=BeautifulSoup("<a> </a>", 'html.parser')
        rsp.extend(a)
        a = p.find_all('a',{'class','adresse pj-lb pj-link'})
        if len(a)==0:
            a=BeautifulSoup("<a> </a>", 'html.parser')
        adrp.extend(a)
        a = p.find_all('a',{'class','activites pj-lb pj-link'})
        if len(a)==0:
            a=BeautifulSoup("<a> </a>", 'html.parser')
        actp.extend(a)
        a = p.find_all('span',{'class','denoms-suppl'})
        if len(a)==0:
            a=BeautifulSoup("<a> </a>", 'html.parser')
        supp.extend(a)
    
    if len(rsp)>0:
        rs = [clean_text(r.text) for r in rsp]
        adresse = [clean_text(r.text) for r in adrp]
        activite = [clean_text(r.text) for r in actp]
        if supp is not None:
            s = [clean_text(su.text) for su in supp]
            s = [re.sub('CONNU SOUS ','',su) for su in s]
        else:    
            s = [""]
    else:
        rs = [""]
        adresse = [""]
        activite = [""]
        s=[""]
        
    test2 = [(row.CABBI,rs[i],adresse[i],activite[i],s[i]) for i in range(min(len(rs),3))]
    print(test2)
    if len(test2)==0 :
        test2 = [(row.CABBI,'','','','')]
    # tri des mots les plus frequents
    if len(testsorted)==0:
        testsorted = test2
    else :
        testsorted.extend(test2)
    
    # arret 5 minutes afin de simuler le comportement d un humain
    time.sleep(5)
    

labels = ['CABBI', 'RS_PJ','ADR_PJ', 'ACT_PJ', 'SP_PJ']
df = pd.DataFrame.from_records(testsorted, columns=labels)

La nouvelle table est exportée au format csv. Les données n'ont finalement pas servi.

In [ ]:
rp.to_csv("/Users/stephaniehimpens/Documents/Hackathon/donnees/data_IlleEtVilaine/rp_enrichi.csv",sep=";")

# Le codage de l'activité (code NAF sur 2 positions) à partir de l'activité déclarée au RP


Import des librairies prérequises :
(TreeTagger doit faire l'objet d'une installation préalable sur le poste de travail)

In [1]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer

import pprint   # For proper print of sequences.
import treetaggerwrapper
#1) build a TreeTagger wrapper:
tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr',TAGDIR='/Users/stephaniehimpens/Documents/teetagger')
 #2) tag your text.

Import du dictionnaire en français :

In [ ]:
import enchant
import unidecode
import pandas as pd

# Correction orthographique

d = enchant.Dict("fr_FR")
# dictionnaire personnel
#d = enchant.request_pwl_dict("/Users/stephaniehimpens/Documents/Hackathon/donnees/dico.txt")

Mise en forme des données du RP sous forme d'une liste de mots nettoyée :

In [2]:
#Import des donnees du RP
don_rp = pd.read_csv('/Users/stephaniehimpens/Documents/Hackathon/donnees/Donnees/rp_final_2014.csv',delimiter=';',encoding='Latin1',low_memory=False)


# Nettoyage des RS du recensement  
don_rp.ACTET_X = [clean_text(word) for word in don_rp.ACTET_X]

# Mise en forme des mots
ACTET_X = [re.sub('[^A-Z\s]|','',str(words)) for words in don_rp.ACTET_X]
ACTET_X = [words.strip() for words in ACTET_X]
# Jetons en langue francaise
tokens = [(nltk.word_tokenize(words,language='french')) for words in ACTET_X]
clean_tokens = []
for token in tokens:
    clean = []
    for c in token :
        if c.lower() in stopwords.words('french') or c=='A' or c=='NAN':
            clean = clean
        else:
            tags = tagger.tag_text(c)
            if len(tags)>0 : c = str(tags[0]).split('\t')[2]
            if d.check(c.upper()) : clean.append(c)
            # partie de recherche dans le dictionnaire trop longue => desactivee
            #else : 
             #   sug = d.suggest(c.upper())
                #if len(sug)>0 : 
                 #   clean.append(sug[0]) 
                #else : clean.append(c)
    clean = [word.upper() for word in clean]
    #if len(clean)>0 : clean = clean[0]
    clean = " ".join(clean)
    clean_tokens.append(clean)
print(clean_tokens[:100])    

# Injection des mots dans la table
test = don_rp
test.ACTET_X = [str(word) for word in clean_tokens]
test = test.loc[test.ACTET_X.isnull()==False]
test = test.loc[test.ACTET_X!='']
test = test.loc[test.ACTET_C.isnull()==False]

# recuperation du numero de departement
categ = [str(ligne)[0:2] for ligne in test.ACTET_C]
# selection des donnees
test = test.loc[[len(text)>1 for text in categ]]
categ = [text for text in categ if len(text)>1]

NameError: name 'pd' is not defined

Séparation de la table en échantillon test et échantillon d'apprentissage :

In [ ]:
# Tirage d un echantillon

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(test['ACTET_X'], categ)

Ajustement d'un modèle SVM :

In [ ]:
# Classification SVM

text_clf_svm = Pipeline([('vect', CountVectorizer()),
              ('tfidf', TfidfTransformer()),
                    ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                       alpha=1e-3, n_iter=10, random_state=42)),
 ])
 

_ = text_clf_svm.fit(X_train, y_train)

Test sur l'échantillon d'apprentissage :

In [ ]:
predicted_svm = text_clf_svm.predict(X_test)
print(np.mean([predicted_svm == y_test]))

# Identification des groupes les plus représentés

Il s'agit dans cette partie de repérer les ensembles de mots revenant souvent. L'idée est qu'il ne faut pas se tromper pour les employeurs appartenant à ces groupes. 

Le travail préliminaire de mise en forme des données textuelles ressemble à celui du paragraphe précédent.
Chargement des librairies requises :

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

Chargement et mise en forme des données du RP (découpage en mots, suppression des mots vides, "taggage" des mots) :

In [ ]:
don_rp = pd.read_csv('/Users/stephaniehimpens/Documents/Hackathon/donnees/Donnees/rp_final_2014.csv',delimiter=';',encoding='Latin1',low_memory=False)

documents = [clean_text(word) for word in don_rp.RS_X]


doc = [re.sub('[^A-Z\s]|','',str(words)) for words in documents]
doc = [words.strip() for words in doc]
tokens = [(nltk.word_tokenize(words,language='french')) for words in doc]
clean_tokens = []
for token in tokens:
    clean = []
    for c in token :
        if c.lower() in stopwords.words('french') or c=='A' or c=='NAN':
            clean = clean
        else:
            tags = tagger.tag_text(c)
            if len(tags)>0 : c = str(tags[0]).split('\t')[2]
            clean.append(c)
            #if d.check(c.upper()) : clean.append(c)
            #else : clean.append(d.suggest(c.upper())[0])
    clean = [word.upper() for word in clean]
    #if len(clean)>0 : clean = clean[0]
    clean = " ".join(clean)
    clean_tokens.append(clean)
print(clean_tokens[:100]) 

Première tentative de clusteration :

In [ ]:
# Premiere version

vectorizer = TfidfVectorizer(ngram_range=(1,2))
X =  vectorizer.fit_transform(cols)

from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(X)
print
print

 
true_k = 5
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

Deuxième tentative (infructueuse) :

In [ ]:
# Latent semantic Analysis
# Nouvel essai
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(clean_tokens)

vectorizer = TfidfTransformer()
X = vectorizer.fit_transform(X_train_counts)

svd = TruncatedSVD(141)
lsa = make_pipeline(svd, Normalizer(copy=False))

X = lsa.fit_transform(X)

# trop de clusters ici => trouver le nombre optimal ?
km = KMeans(n_clusters=141, init='k-means++', max_iter=100)
km.fit(X)
model=km

print(X.shape)

Visualisation des clusters obtenus :

In [ ]:
# Visualisation des clusters
clusters = model.labels_.tolist()
 
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print
 
 
print("\n")
print("Prediction")

Un programme sommaire d'appariement des données a également été écrit mais il n'est pas particulièrement intéressant et il n'a pas été reproduit ici. 